In [1]:
import pandas as pd
import preprocess
import regression
import numpy as np

In [2]:
test_data = pd.read_csv('./data/test set.csv')
train_data = pd.read_csv('./data/trainSet.csv')

In [3]:
list(test_data), len(test_data)

(['F1',
  'F2',
  'F3',
  'F4',
  'F5',
  'F6',
  'F7',
  'F8',
  'F9',
  'F10',
  'F11',
  'F12',
  'F13',
  'F14',
  'F15',
  'F16',
  'F17',
  'F18',
  'F19',
  'F20',
  'F21',
  'F22',
  'F23',
  'F24',
  'F25',
  'F26',
  'F27',
  'F28',
  'F29',
  'F30',
  'F31',
  'F32'],
 183267)

In [4]:
list(train_data), len(train_data)

(['F1',
  'F2',
  'F3',
  'F4',
  'F5',
  'F6',
  'F7',
  'F8',
  'F9',
  'F10',
  'F11',
  'F12',
  'F13',
  'F14',
  'F15',
  'F16',
  'F17',
  'F18',
  'F19',
  'F20',
  'F21',
  'F22',
  'F23',
  'F24',
  'F25',
  'F26',
  'F27',
  'F28',
  'F29',
  'F30',
  'F31',
  'F32',
  'label'],
 200000)

In [5]:
train_x = train_data.values[:, :-1]
train_target = train_data.values[:, -1].reshape(-1, 1)

In [41]:
z_scale_processed_x, mean, std = preprocess.z_score_scaling(train_x)
min_max_scaled_x = preprocess.min_max_scaling(train_x)
processed_test, _, _ = preprocess.z_score_scaling(test_data.values, mean, std)


In [ ]:
# Feature Discretization with kmeans

In [8]:
# trainer is used for training and should return the trained parameters,
# tester should receive x and the parameters and return the predicted item
# args and kwargs are extra parameters to pass to the trainer.
def k_fold(k, trainer, tester, x, target, *args, **kwargs):
    samples, features = x.shape
    fold_size = int(samples / k)
    shuffled = np.hstack((x, target))
    np.random.shuffle(shuffled)
    shuffled_x = shuffled[:, :-1]
    shuffled_target = shuffled[:, -1].reshape(-1, 1)
    x_sets = np.array([shuffled_x[i * fold_size : (i+1) * fold_size] for i in range(k)])
    target_sets = np.array([shuffled_target[i * fold_size: (i+1) * fold_size] for i in range(k)])
    precisions_sum = 0
    for i in range(k):
        training_set = np.empty(shape=(0, features))
        training_targets = np.empty(shape=(0, 1))
        test_set = x_sets[i]
        test_targets = target_sets[i]
        for j in range(k):
            if j != i:
                training_set = np.concatenate((training_set, x_sets[j]))
                training_targets = np.concatenate((training_targets, target_sets[j]))
        the = trainer(training_set, training_targets, *args, **kwargs)
        predicted = tester(test_set, the)
        predicted = [1 if e > 0.5 else 0 for e in predicted]
        corrects = 0
        for j in range(fold_size):
            if predicted[j] == test_targets[j]:
                corrects += 1
        print('%f at k=%d' % (corrects / fold_size, i))
        precisions_sum += corrects / fold_size
    result = precisions_sum / k
    print('Average: %f' % result)
    return result

In [43]:
iters = 500
lr = 0.2
k_fold(5, regression.regression, regression.logistic_h, z_scale_processed_x, train_target, 
       regression.logistic_h, iters, lr, lbd=5.0, log=True, logInterval=125)

2.7527275255038584
0.6638893206855518
0.6612073162483364
0.6607079106337724
0.608950 at k=0
3.149816251970812
0.6645556303203741
0.6601965682863565
0.6596383002253249
0.606525 at k=1
2.733415425865455
0.6650308376778666
0.6610839430332004
0.6604200562860225
0.606775 at k=2
2.734035519649418
0.6678951741369885
0.6610448210937401
0.6603694250840146
0.611200 at k=3
2.957042688412527
0.6665823017294774
0.6610789340522618
0.6598974979143948
0.603100 at k=4
Average: 0.607310


0.6073099999999999

In [10]:
def shuffle(x, label):
    shuffled = np.hstack((x, label))
    np.random.shuffle(shuffled)
    shuffled_x = shuffled[:, :-1]
    shuffled_target = shuffled[:, -1].reshape(-1, 1)
    return shuffled_x, shuffled_target

In [11]:
iters = 500
lr = 0.2
shuffled_x, shuffled_label = shuffle(z_scale_processed_x, train_target)
the = regression.regression(z_scale_processed_x, train_target, regression.logistic_h, iters, lr, lbd=5.0, log=True)

2.6239971200185224
0.6660982703355481
0.6611997658892804
0.6604064398202419
0.6601171238307323


In [12]:
result = regression.logistic_h(z_scale_processed_x, the)

In [13]:
result = [1 if e > 0.5 else 0 for e in result]

In [14]:
accurate = 0
for i in range(len(result)):
    if result[i] == train_target[i]:
        accurate += 1
accurate

121488

In [15]:
result = regression.logistic_h(processed_test, the)
pd.DataFrame([[i+1, 1 if result[i] > 0.5 else 0] for i in range(len(result))]).to_csv('result.csv', header=['Id', 'Predicted'], index=False)

In [16]:
import torch
from torch import functional as F

In [70]:
class Net(torch.nn.Module):
    def __init__(self, features, hidden1, hidden2, out):
        super(Net, self).__init__()
        self.layer1 = torch.nn.Linear(features, hidden1)
        self.layer2 = torch.nn.Linear(hidden1, hidden2)
        self.layer3 = torch.nn.Linear(hidden2, out)
        
    def forward(self, x):
        o1 = self.layer1(x)
        o2 = self.layer2(o1)
        return self.layer3(o2)
    

In [71]:
net = Net(z_scale_processed_x.shape[1], 260, 127, 2)
net

Net(
  (layer1): Linear(in_features=32, out_features=260, bias=True)
  (layer2): Linear(in_features=260, out_features=127, bias=True)
  (layer3): Linear(in_features=127, out_features=2, bias=True)
)

In [72]:
optimizer = torch.optim.Adam(net.parameters())
loss_func = torch.nn.CrossEntropyLoss()

In [73]:
epoch = 20
def trainer(x, target):
    for i in range(epoch):
        o = net(torch.Tensor(x))
        loss = loss_func(o, torch.Tensor(target).long().squeeze(1))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

def tester(x, *args):
    o = net(torch.Tensor(x))
    return torch.max(o, 1)[1].numpy().reshape(-1, 1)

In [74]:
k_fold(5, trainer, tester, z_scale_processed_x, train_target)

0.607600 at k=0
0.606700 at k=1
0.606775 at k=2
0.606200 at k=3
0.607800 at k=4
Average: 0.607015


0.607015